In [1]:
import sys
import os
import operator
from itertools import *

sys.path.append('/home/domain/data/prog/pyrosetta/')

from rosetta import *
from toolbox import *
from rosetta.utility import vector1_bool
init() 

PYROSETTA_DATABASE environment variable was set to: /home/domain/data/prog/pyrosetta/rosetta_database; using it....
PyRosetta 3.4.0 [Rosetta-3.4 55980:55980:fc2b8dc1c719421d421c7a90f3e684ca0c1d0609] retrieved from: 
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Rosetta version  from
core.init: command: PyRosetta -ex1 -ex2aro -database /home/domain/data/prog/pyrosetta/rosetta_database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-1941733889 seed_offset=0 real_seed=-1941733889
core.init.random: RandomGenerator:init: Normal mode, seed=-1941733889 RG_type=mt19937


In [2]:
os.chdir('/home/domain/data/kirill/AB/XOP/rosetta')

# Input molecules options

In [3]:
pdb_filename = "reactive21c.pdb"
partners = "A_X"
ligand_params = Vector1(['/home/domain/data/kirill/AB/XOP/prepare_ligs/XOPR.params'])

In [4]:
scorefxn = create_score_function("talaris2013_cst")
scorefxn_low = create_score_function("interchain_cen")
scorefxn_high = create_score_function_ws_patch("talaris2013","docking")

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/sp2_elec_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/rama/Rama_smooth_dyn.dat_ss_6.4
basic.io.database: Database file opened: scoring/score_functions/P_AA_pp/P_AA
basic.io.database: Database file opened: scoring/score_functions/P_AA_pp/P_AA_n
basic.io.database: Database file opened: scoring/score_functions/P_AA_pp/P_AA_p

# run once

In [5]:
new_res_set  = ChemicalManager.get_instance().nonconst_residue_type_set('fa_standard')
atoms    = ChemicalManager.get_instance().atom_type_set('fa_standard')
mm_atoms = ChemicalManager.get_instance().mm_atom_type_set('fa_standard')
orbitals = ChemicalManager.get_instance().orbital_type_set('fa_standard')
elements = ChemicalManager.get_instance().element_set('fa_standard')
new_res_set.read_files(ligand_params, atoms, elements, mm_atoms, orbitals)


core.chemical.ResidueTypeSet: Finished initializing fa_standard residue type set.  Created 9282 residue types


In [6]:
pose = Pose()
pose_from_pdb(pose, new_res_set, pdb_filename)

core.io.pdb.file_data: [ WARNING ] discarding 3 atoms at position 1 in file reactive21c.pdb. Best match rsd_type:  VAL_p:NtermProteinFull
core.io.pdb.file_data: [ WARNING ] discarding 4 atoms at position 2 in file reactive21c.pdb. Best match rsd_type:  GLN
core.io.pdb.file_data: [ WARNING ] discarding 2 atoms at position 3 in file reactive21c.pdb. Best match rsd_type:  LEU
core.io.pdb.file_data: [ WARNING ] discarding 4 atoms at position 4 in file reactive21c.pdb. Best match rsd_type:  GLN
core.io.pdb.file_data: [ WARNING ] discarding 4 atoms at position 5 in file reactive21c.pdb. Best match rsd_type:  GLU
core.io.pdb.file_data: [ WARNING ] discarding 2 atoms at position 6 in file reactive21c.pdb. Best match rsd_type:  SER
core.io.pdb.file_data: [ WARNING ] discarding 2 atoms at position 7 in file reactive21c.pdb. Best match rsd_type:  GLY
core.io.pdb.file_data: [ WARNING ] discarding 6 atoms at position 8 in file reactive21c.pdb. Best match rsd_type:  PRO
core.io.pdb.file_data: [ WARN

# Moovers

In [7]:
import rosetta.protocols.rigid as rigid_moves
jump_num = 1

In [8]:
pert_mover = rigid_moves.RigidBodyPerturbMover(jump_num, 8,0.5)
rand_mover = rigid_moves.RigidBodyRandomizeMover(pose, jump_num, rigid_moves.partner_upstream)
slide = FaDockingSlideIntoContact(jump_num)

In [20]:
pymover = PyMOL_Mover()
pymover.link.udp_ip = "10.0.0.15"
pymover.link.udp_port = 7001
pymover.keep_history(True)

In [10]:
movemap = MoveMap()
movemap.set_jump(jump_num, True)
min_mover = MinMover()
min_mover.movemap(movemap)
min_mover.score_function(scorefxn) 
scorefxn(pose)

In [ ]:
PackMover

# Docking

In [17]:
dock_lowres = DockingLowRes(scorefxn_low,jump_num)

In [32]:
dock_hires = DockMCMProtocol()
dock_hires.set_scorefxn(scorefxn_high)
dock_hires.set_partners("A_B")

# Application

In [46]:
print scorefxn(pose)

459.81796146


In [12]:
pymover.send_energy(pose, fa_atr)
pymover.apply(pose)

In [85]:
# the output foes to terminal, so we need to capture it
o = rosetta.utility.OStringStream()
en = scorefxn.show(o,pose)
print o.str()

------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       0.800   -1801.890   -1441.512
 fa_rep                       0.440     544.451     239.559
 fa_sol                       0.750    1094.810     821.108
 fa_intra_rep                 0.004     657.334       2.629
 fa_elec                      0.500    -229.132    -114.566
 pro_close                    1.000     122.218     122.218
 hbond_sr_bb                  1.170     -23.006     -26.917
 hbond_lr_bb                  1.170    -114.542    -134.014
 hbond_bb_sc                  1.170     -29.051     -33.990
 hbond_sc                     1.100     -32.214     -35.436
 dslf_fa13                    1.000       0.000       0.000
 atom_pair_constraint         1.000       0.000       0.000
 coordinate_constraint        1.000       0.000       0.000
 angle_constraint             1.000  

In [86]:
o = rosetta.utility.OStringStream()
en = pose.energies().show(o)
print o.str()

E               fa_atr        fa_rep        fa_sol  fa_intra_rep       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13atom_pair_conscoordinate_conangle_constraidihedral_const          rama         omega        fa_dun       p_aa_pp           ref    chainbreakres_type_const
E(i)   1         -2.92          0.52          1.78          3.57         -0.39          0.00          0.00          0.00         -0.17          0.00          0.00          0.00          0.00          0.00          0.00          0.00          0.05          1.75          0.00          0.74          0.00          0.00
E(i)   2         -3.28          0.35          2.30          1.05         -0.53          0.00          0.00          0.00          0.00          0.00          0.00          0.00          0.00          0.00          0.00         -0.67          0.18          5.90         -0.08         -1.18          0.00          0.00
E(i)   3         -4.77          0.37          1.9

In [ ]:
Fa

In [30]:
for a in range(10):
    dock_lowres.apply(pose)
    pymover.apply(pose)
    print scorefxn(pose)
   

459.81796146
10924.044376
10160.1215865
7188.0148449
7188.0148449
7955.18719327
7955.18719327
7955.18719327
7955.18719327
7955.18719327


In [34]:
for a in range(10):
    dock_hires.apply(pose)
    pymover.apply(pose)
    print scorefxn(pose)

339.051996315
336.833691797
336.833218178
336.832747498
336.832279746
336.831814907
336.831352968
335.659607764
332.467747243
333.75655582


In [ ]:
setup_foldtree(pose, "A_X", Vector1([2]))

In [76]:
print pose.fold_tree()
pymover.apply(pose)

FOLD_TREE  EDGE 1 257 -1  EDGE 257 432 -1  EDGE 257 433 1 



In [77]:

print pose.jump(jump_num).get_rotation()
print pose.jump(jump_num).get_translation()

   -0.02218838805249174     -0.9844282194336131     -0.1743810661121046
     0.9847842247706724    -0.05159210637134359      0.1659466335994563
    -0.1723592355473201     -0.1680456347035848      0.9705941265939235

     -10.12947180280834      -8.189185815435053       25.73146869901586


In [78]:
pert_mover.apply(pose)
pymover.apply(pose)

In [46]:
print pose.jump(jump_num).get_rotation()
print pose.jump(jump_num).get_translation()

    -0.2028955645824245     -0.7478458109642226     -0.6321076118004272
     0.9482282372383681     -0.3111293122484540     0.06373194774694470
    -0.2443288766736258     -0.5864513569624233      0.7722552725234997

     -8.607245227091774      -13.04616946212963       28.21144823355416
